In [0]:

dbutils.fs.mount(
    source='wasbs://srini-adlsgen2adventureworks@adlsgen2adventureworks.blob.core.windows.net/',
    mount_point='/mnt/bronze-srini',
    extra_configs={
'fs.azure.account.key.adlsgen2adventureworks.blob.core.windows.net': 'KFrirnuNDTcvixYNQIlGHiDoMLPrOeh5VvED866hfXh6Vn6OEFJXLGaSpHDTWzTp2ir503LwgUSu+ASt7YmK+w=='
    }
)

In [0]:

PD_PATH = "/mnt/bronze-srini/BRONZE"

def list_files_recursively(directory):
    file_paths = []
   
    files = dbutils.fs.ls(directory)
    for file in files:
        if file.isDir():
            # If the item is a directory, recursively call the function
            file_paths.extend(list_files_recursively(file.path))
        else:
            # If the item is a file, append its path to the list
            file_paths.append(file.path)
    return file_paths
 
# Get list of all files in the parent directory and its subdirectories
file_paths = list_files_recursively(PD_PATH )
print(file_paths)

['dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Department/2024-02-21T10:59:31.7566414/HumanResources_Department.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Employee/2024-02-21T10:59:51.2337011/HumanResources_Employee.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/EmployeeDepartmentHistory/2024-02-21T11:00:04.9490793/HumanResources_EmployeeDepartmentHistory.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/EmployeePayHistory/2024-02-21T11:00:19.5488464/HumanResources_EmployeePayHistory.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/JobCandidate/2024-02-21T11:00:35.0597455/HumanResources_JobCandidate.csv', 'dbfs:/mnt/bronze-srini/BRONZE/HumanResources/Shift/2024-02-21T11:00:52.2700202/HumanResources_Shift.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/Address/2024-02-21T11:01:07.2385191/Person_Address.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/AddressType/2024-02-21T11:01:37.4855778/Person_AddressType.csv', 'dbfs:/mnt/bronze-srini/BRONZE/Person/BusinessEntity/2024-02-21

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
from datetime import datetime 
import datetime,pytz
from pyspark.sql.functions import substring, col, lit, concat, expr
for file in file_paths:

    df = spark.read.csv(file, header=True, inferSchema=True)
    print(df)
    for column in df.columns:
        
        def encode_to_utf8(text):     
         return text.encode('utf-8') 
         df=df.withColumn(column,encode_to_utf8(column))
        if df.schema[column].dataType == StringType():
            df = df.withColumn(column,upper(df[column]))
        if df.schema[column].dataType == DecimalType() or df.schema[column].dataType == DoubleType():
            df.withColumn(column, format_number(df[column],2))
        if column == 'AverageRate' or column == 'EndOfDayRate':
            df.withColumn(column, floor(df[column]*75))
        fp= 'dbfs:/mnt/bronze-srini/BRONZE/Production/Product/2024-02-21T11:03:01.3756098/Production_Product.csv'
        if file ==fp:
            df = spark.read.csv(file, header=True, inferSchema=True)  
        if column == 'ProductNumber':
            df = df.withColumn('ProductNumber', concat(col('Name'), substring(col('ProductNumber'), 3, 100)))

        f_p= 'dbfs:/mnt/bronze-srini/BRONZE/Person/CountryRegion/2024-02-21T11:03:01.3756098/Person_CountryRegion.csv'  
        if file ==f_p:
            df = spark.read.csv(file, header=True, inferSchema=True)
        if column == 'Name':   
            df=df.withColumn('New_Region_Name', upper(substring('Name', 1,3)))
               # df.show()
    
    current_timestamp = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%dT%H-%M-%S.%f')[:-3]
 
      # Split the file_path string based on '/'
    path_parts = file.split('/')
 
     # Replace 'Bronze' with 'Silver' in the path_parts list
    path_parts[path_parts.index('BRONZE')] = 'Silver1'
 
     # Replace the timestamp part with the current timestamp
    path_parts[-2] = current_timestamp
 
     # Join the path_parts list back into a string using '/'
    dynamic_path = '/'.join(path_parts)
    df.coalesce(1).write.csv(dynamic_path,header=True, mode="overwrite")

DataFrame[DepartmentID: int, Name: string, GroupName: string, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, NationalIDNumber: int, LoginID: string, OrganizationNode: string, OrganizationLevel: int, JobTitle: string, BirthDate: timestamp, MaritalStatus: string, Gender: string, HireDate: timestamp, SalariedFlag: boolean, VacationHours: int, SickLeaveHours: int, CurrentFlag: boolean, rowguid: string, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, DepartmentID: int, ShiftID: int, StartDate: timestamp, EndDate: timestamp, ModifiedDate: timestamp]
DataFrame[BusinessEntityID: int, RateChangeDate: timestamp, Rate: double, PayFrequency: int, ModifiedDate: timestamp]
DataFrame[JobCandidateID: string, BusinessEntityID: string, Resume: string, ModifiedDate: string]
DataFrame[ShiftID: int, Name: string, StartTime: timestamp, EndTime: timestamp, ModifiedDate: timestamp]
DataFrame[AddressID: int, AddressLine1: string, AddressLine2: string, City: string, StateProvinceID: int, Po